In [2]:
import os
import io
import requests
import zipfile
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def download_day(date, nodes=None):
    """Download and extract DAM_LMP CSV for a single day"""
    base_url = "http://oasis.caiso.com/oasisapi/SingleZip"
    date_str = date.strftime("%Y%m%d")
    
    params = {
        "resultformat": "6",
        "queryname": "PRC_LMP",
        "version": "12",
        "market_run_id": "DAM",
        "startdatetime": f"{date_str}T08:00-0000",
        "enddatetime": f"{date_str}T08:00-0000",
        "grp_type": "ALL"
    }

    try:
        response = requests.get(base_url, params=params, timeout=60)
        response.raise_for_status()
    except Exception as e:
        print(f"❌ Download error for {date_str}: {e}")
        return None

    try:
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            for name in z.namelist():
                if "DAM_LMP" in name and name.endswith(".csv"):
                    with z.open(name) as f:
                        df = pd.read_csv(f)
                        if nodes is not None:
                            df = df[df["NODE"].isin(nodes)]
                        return df
    except Exception as e:
        print(f"❌ ZIP processing error for {date_str}: {e}")
        return None

def fetch_caiso_dam_lmp_parallel(start_date, end_date, nodes=None, max_workers=8):
    """Parallel downloader for CAISO DAM_LMP data"""
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")
    date_list = [start_dt + timedelta(days=i) for i in range((end_dt - start_dt).days + 1)]

    all_data = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(download_day, date, nodes): date for date in date_list}
        for future in tqdm(as_completed(futures), total=len(futures), desc="CAISO Parallel"):
            result = future.result()
            if result is not None:
                all_data.append(result)

    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

# Example usage
if __name__ == "__main__":
    df = fetch_caiso_dam_lmp_parallel(
        start_date="2023-01-01",
        end_date="2024-01-01",  # Or None for all nodes
        max_workers=6  # Tune based on your internet speed / CPU
    )

    print(df.head())
    df.to_csv("caiso_dam_lmp_parallel.csv", index=False)


CAISO Parallel:   0%|                          | 4/1201 [00:00<03:05,  6.47it/s]

❌ Download error for 20220104: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20220104T08%3A00-0000&enddatetime=20220104T08%3A00-0000&grp_type=ALL❌ Download error for 20220106: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20220106T08%3A00-0000&enddatetime=20220106T08%3A00-0000&grp_type=ALL

❌ Download error for 20220105: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20220105T08%3A00-0000&enddatetime=20220105T08%3A00-0000&grp_type=ALL
❌ Download error for 20220102: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/GroupZip?groupid=DAM_LMP_GRP&startdatetime=20220102T08:00-0000&resultformat=6&version=12


CAISO Parallel:   1%|▏                         | 8/1201 [00:01<02:54,  6.83it/s]

❌ Download error for 20220108: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20220108T08%3A00-0000&enddatetime=20220108T08%3A00-0000&grp_type=ALL❌ Download error for 20220107: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20220107T08%3A00-0000&enddatetime=20220107T08%3A00-0000&grp_type=ALL
❌ Download error for 20220109: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20220109T08%3A00-0000&enddatetime=20220109T08%3A00-0000&grp_type=ALL

❌ Download error for 20220110: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20220110T08%3

CAISO Parallel:   1%|▏                        | 12/1201 [00:02<02:50,  6.97it/s]

...

❌ Download error for 20250411: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20250411T08%3A00-0000&enddatetime=20250411T08%3A00-0000&grp_type=ALL
❌ Download error for 20250412: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20250412T08%3A00-0000&enddatetime=20250412T08%3A00-0000&grp_type=ALL


CAISO Parallel: 100%|██████████████████████▉| 1198/1201 [03:12<00:00,  6.13it/s]

❌ Download error for 20250413: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20250413T08%3A00-0000&enddatetime=20250413T08%3A00-0000&grp_type=ALL
❌ Download error for 20250414: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20250414T08%3A00-0000&enddatetime=20250414T08%3A00-0000&grp_type=ALL
❌ Download error for 20250415: 429 Client Error: Too Many Requests for url: https://oasis.caiso.com/oasisapi/SingleZip?resultformat=6&queryname=PRC_LMP&version=12&market_run_id=DAM&startdatetime=20250415T08%3A00-0000&enddatetime=20250415T08%3A00-0000&grp_type=ALL


CAISO Parallel: 100%|███████████████████████| 1201/1201 [03:21<00:00,  5.97it/s]


       INTERVALSTARTTIME_GMT        INTERVALENDTIME_GMT      OPR_DT  OPR_HR  \
0  2022-01-01T08:00:00-00:00  2022-01-01T09:00:00-00:00  2022-01-01       1   
1  2022-01-01T16:00:00-00:00  2022-01-01T17:00:00-00:00  2022-01-01       9   
2  2022-01-02T05:00:00-00:00  2022-01-02T06:00:00-00:00  2022-01-01      22   
3  2022-01-01T22:00:00-00:00  2022-01-01T23:00:00-00:00  2022-01-01      15   
4  2022-01-01T15:00:00-00:00  2022-01-01T16:00:00-00:00  2022-01-01       8   

   OPR_INTERVAL    NODE_ID_XML        NODE_ID           NODE MARKET_RUN_ID  \
0             0  0096WD_7_N001  0096WD_7_N001  0096WD_7_N001           DAM   
1             0  0096WD_7_N001  0096WD_7_N001  0096WD_7_N001           DAM   
2             0  0096WD_7_N001  0096WD_7_N001  0096WD_7_N001           DAM   
3             0  0096WD_7_N001  0096WD_7_N001  0096WD_7_N001           DAM   
4             0  0096WD_7_N001  0096WD_7_N001  0096WD_7_N001           DAM   

  LMP_TYPE XML_DATA_ITEM  PNODE_RESMRID GRP_TYPE  POS   